In [1]:
from os import getcwd
import os.path as op
from glob import glob

In [2]:
from clients.utils import create_allocation_table
from hardware.plates import create_plates_from_allocation_table

expt_name = 'A81_E214'
allocation_table = create_allocation_table(expt_name)
expt_plates = create_plates_from_allocation_table(allocation_table)

In [3]:
from hardware.qpcr import QpcrDataFile

id_qpcr_plate = 'A81_E214_1_ID'
cwd = getcwd()
nav_root = op.abspath(op.join(cwd, op.pardir,
                              op.pardir, op.pardir))
data_folder = op.join(nav_root, 'hardware', 'tests', 'data')
f = op.join(data_folder, id_qpcr_plate + '.xls')

qpcr_data = QpcrDataFile(f)
id_qpcr_inst_plate = qpcr_data.get_data_by_well()

C:\Users\chris.icely\AppData\Local\conda\conda\envs\jupyter\lib\site-packages\pandas\core\ops.py:816: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(x, name)(y)


In [4]:
from hardware.labchip import LabChip

lc_plate = '20180103_A'
fds = glob(op.join(data_folder, '*' + lc_plate + '*.csv'))

files = {}
for f in fds:
    if 'PeakTable' in f:
        files['peak_file'] = f
    elif 'RawTable' in f:
        files['raw_file'] = f
    elif 'WellTable' in f:
        
        files['well_file'] = f
lc = LabChip(**files)
lc_inst_plate = lc.get_data_by_well()

In [5]:
assays = {'Ca_rpb7_x.1_Cal04_Cal03': 215,
          'Cg_rps0_x.1_Cgl03_Cgl04': 217,
          'Ab_pgaD_x.10_Aba38_Aba42': 225,
          'Ec_uidA_x.2_Eco64_Eco66': 240}

In [7]:
from clients.expt_recipes.common.model_builders import build_id_qpcr_constituents
from clients.expt_recipes.nested.models import IdConstituents
from clients.expt_recipes.nested.model_builders import (build_id_qpcr_datas_from_inst_data,
                                                        build_labchip_datas_from_inst_data,
                                                        create_nested_groupings,
                                                        get_wells_by_id_assay)
from clients.expt_recipes.nested.utils import (get_id_qpcr_plate_names,
                                               get_labchip_plate_names,
                                               create_id_qpcr_lc_mapping,
                                               get_lc_dilutions)
from clients.expt_recipes.nested.models import NestedMasterTable

In [8]:
%%time
id_plate_constituents = build_id_qpcr_constituents(
    expt_plates[id_qpcr_plate], expt_plates, IdConstituents)

wells_by_id_assay = get_wells_by_id_assay(id_plate_constituents)

id_qpcr_datas = build_id_qpcr_datas_from_inst_data(id_plate_constituents,
                                                   id_qpcr_inst_plate)

Wall time: 9.53 ms


In [9]:
%%time
mapping = create_id_qpcr_lc_mapping(expt_plates[lc_plate])
dilutions = get_lc_dilutions(expt_plates[lc_plate])
lc_datas = build_labchip_datas_from_inst_data(
    id_plate_constituents, lc_inst_plate, mapping, assays, dilutions)

Wall time: 2.51 ms


In [10]:
%%time
qwells = wells_by_id_assay[('Cg_rps0_x.1_Cgl03_Cgl04',)]
lcwells = [mapping[qw] for qw in qwells]

nmt = NestedMasterTable.create_from_models(qwells,
                                           id_qpcr_plate,
                                           lcwells,
                                           lc_plate,
                                           id_plate_constituents,
                                           id_qpcr_datas,
                                           lc_datas)

Wall time: 12 ms


In [11]:
import pandas as pd
df = pd.DataFrame(nmt.rows)
df

,qPCR well,LC well,PA Assay Name,PA Assay Conc.,PA template Name,PA Template Conc.,PA Human Name,PA Human Conc.,ID Assay Name,ID Assay Conc.,...,Tm2,Tm3,Tm4,Tm Specif,Tm NS,Tm PD,Specif ng/ul,NS ng/ul,PD ng/ul,PA Template Name
0,A81_E214_1_ID_F02,20180103_A_N01,"(poolE1,)","(1.0,)",None,"(5.0,)","(HgDNA_Promega287320,)","(3000.0,)","(Cg_rps0_x.1_Cgl03_Cgl04,)","(0.4,)",...,NaN,NaN,NaN,True,False,False,70.680228,0.000000,10.894672,"(C.glabrata_15545,)"
1,A81_E214_1_ID_E02,20180103_A_M01,"(poolE1,)","(1.0,)",None,"(5.0,)",(),(),"(Cg_rps0_x.1_Cgl03_Cgl04,)","(0.4,)",...,NaN,NaN,NaN,True,False,False,63.456174,0.000000,9.210992,"(C.glabrata_15545,)"
2,A81_E214_1_ID_D02,20180103_A_L01,"(poolE1,)","(1.0,)",None,"(5.0,)",(),(),"(Cg_rps0_x.1_Cgl03_Cgl04,)","(0.4,)",...,NaN,NaN,NaN,True,False,False,61.659857,0.000000,0.000000,"(C.glabrata_15545,)"
3,A81_E214_1_ID_G02,20180103_A_O01,"(poolE1,)","(1.0,)",None,"(5.0,)","(HgDNA_Promega287320,)","(3000.0,)","(Cg_rps0_x.1_Cgl03_Cgl04,)","(0.4,)",...,NaN,NaN,NaN,True,False,False,52.116491,0.000000,8.605230,"(C.glabrata_15545,)"
4,A81_E214_1_ID_H02,20180103_A_P01,"(poolE1,)","(1.0,)",None,"(5.0,)","(HgDNA_Promega287320,)","(3000.0,)","(Cg_rps0_x.1_Cgl03_Cgl04,)","(0.4,)",...,89.431160,NaN,NaN,True,True,False,60.103285,0.000000,9.595831,"(C.glabrata_15545,)"
5,A81_E214_1_ID_C02,20180103_A_K01,"(poolE1,)","(1.0,)",None,"(5.0,)",(),(),"(Cg_rps0_x.1_Cgl03_Cgl04,)","(0.4,)",...,NaN,NaN,NaN,True,False,False,85.793943,0.000000,0.000000,"(C.glabrata_15545,)"
6,A81_E214_1_ID_A02,20180103_A_I01,"(poolE1,)","(1.0,)",None,(),(),(),"(Cg_rps0_x.1_Cgl03_Cgl04,)","(0.4,)",...,79.028015,NaN,NaN,False,False,True,0.000000,0.000000,6.739206,()
7,A81_E214_1_ID_B02,20180103_A_J01,"(poolE1,)","(1.0,)",None,(),"(HgDNA_Promega287320,)","(3000.0,)","(Cg_rps0_x.1_Cgl03_Cgl04,)","(0.4,)",...,NaN,NaN,NaN,False,True,False,0.000000,0.000000,10.356878,()
8,A81_E214_1_ID_F10,20180103_A_N05,(),(),None,(),(),(),"(Cg_rps0_x.1_Cgl03_Cgl04,)","(0.4,)",...,NaN,NaN,NaN,True,False,False,113.775259,0.000000,0.000000,()
9,A81_E214_1_ID_D10,20180103_A_L05,(),(),None,(),(),(),"(Cg_rps0_x.1_Cgl03_Cgl04,)","(0.4,)",...,NaN,NaN,NaN,True,False,False,41.243288,0.000000,0.000000,()
